In [11]:
import sys
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import urllib.request
from konlpy.tag import Okt

import json

tokenizer = Tokenizer()

# wordInext 목록 받아오기
with open('LSTM_module_ver2.0_wordIndex.json') as f:
    word_index = json.load(f)
    tokenizer.word_index = word_index
    
with open('LSTM_module_ver2.0_variable.json') as f:
    var = json.load(f)
    
max_data = var['max_data'] # 데이터 문자열 최대 길이
min_data = var['min_data'] # 데이터 문자열 최소 길이
max_len = var['max_len'] # 전체 데이터 셋 길이 설정 (메일의 최대 길이)

model = tf.keras.models.load_model('LSTM_module_ver2.0.h5')
model.summary()

okt = Okt()

def preTreatment(data):
    stopwords = ['\n','.','?',',','']
    temp = okt.morphs(data, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 처리
    return temp


def sentiment_predict(new_sentence):
    data = tokenizer.texts_to_sequences([new_sentence]) # 단어를 숫자값, 인덱스로 변환하여 저장
    print(data)
    pad_new = pad_sequences(data, maxlen = max_len) # 모든 메일의 길이를 100로 설정 (빈 부분은 0으로 패딩)
    score = float(model.predict(pad_new))
    if (score > 0.5):
        print(' {:.2f}% 확률로 스팸입니다.'.format(score * 100))
    else:
        print(' {:.2f}% 확률로 스팸이 아닙니다.'.format((1 - score) * 100))
    
    print('-------------------------------------------------------------------------')
        

# 음성파일 STT 변환
import speech_recognition as sr

r = sr.Recognizer()
harvard = sr.AudioFile('spam.wav') # 10MB 용량 제한
with harvard as source:
    audio = r.record(source)

t = r.recognize_google(audio, language='ko-KR')

temp_X = []
x = [t[i:i+max_data] for i in range(0, len(t), max_data)] # 텍스트 파일 중 150 길이로 데이터 길이 제한
for s in x:
    if len(s) > min_data: # 길이가 50 이하인 데이터 삭제
        temp_X.append(s)
x = temp_X
for txt in x:
    print(txt)
    data = preTreatment(txt)
    sentiment_predict(data)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         2468100   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,585,477
Trainable params: 2,585,477
Non-trainable params: 0
_________________________________________________________________
예 잘 따른 본인 살 거니까 사건 경희대 좀 설명 드릴 건데 네이버 카페 중고나라 사이클 계시나요 안녕하세요 여러분 강성민입니다 오늘은 보이스피싱 실제 통화했던 내용을 들려 드릴 건데요 실제로 제가 겪었던 일이고요 뒤에 가서 살짝 역관광 하는 부분도 있으니까 영상 끝까
[[141, 34, 1017, 69, 276, 30, 143, 210, 54, 817, 573, 563, 1359, 430, 1698, 16734, 384, 1645, 4735, 3067, 65, 56, 21, 2864, 1235, 530, 1, 377, 20, 87, 573, 563, 79, 1235, 44, 96, 4, 2301, 479, 814, 567, 6, 29, 1124, 1117, 5